# <center>Test book

# Imports

In [21]:
from os import listdir
from os.path import isdir, isfile

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import pymupdf

from sentence_transformers import SentenceTransformer, util
from transformers import AutoModel, AutoTokenizer

import faiss

## Read the data - Text Extraction

In [83]:
path = "BOOKS/"
document_paths = []
titles = []
documents = []
counter = 0

# ToDo Recursively
for item in listdir(path):
    # print(item)
    counter +=1
    txt = pymupdf.get_text(path + item)
    if txt:
        assert len(txt) == 1
        titles.append(item)
        documents.append(txt[0])
    else:
        print(f"ERROR on file: {item}")
    if counter == 4: break

df = pd.DataFrame({"title" : titles, "document" : documents})
print(df.shape[0])
df

4


,title,document
0,L'Intelligence artificielle pour les Nuls - Jo...,I\nChapitre 2\nDéfinir le rôle des données\nDA...
1,L'Intelligence artificielle pour les Nuls - Jo...,"humains, mais ces chiffres sont déjà remis en ..."
2,L'Intelligence artificielle pour les Nuls - Jo...,cessé d’être simplement de l’information trait...
3,L'Intelligence artificielle pour les Nuls - Jo...,"logicielles de Google parcourent le Web, saute..."


## Load model

In [ ]:
model = SentenceTransformer(r"C:\Users\Utilisateur\Documents\mytheque\Models\sentence-camembert-large", local_files_only=True)

## Text embedding

In [84]:
def embedding(txt):
    temp = model.encode(txt)
    print(type(temp))
    print(type(temp.tolist()))
    print(type(temp.item(0)))
    return temp
    return temp.tolist()

df['embedding'] = df['document'].apply(embedding)
df['embedding'] = df['embedding'].apply(lambda emb : emb.tolist())
df['embedding']

<class 'numpy.ndarray'>
<class 'list'>
<class 'float'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'float'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'float'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'float'>


0    [-0.25839850306510925, -0.22478339076042175, -...
1    [-0.2057669311761856, -0.6190314888954163, -0....
2    [-0.43562543392181396, -0.5640958547592163, 0....
3    [-0.1016184613108635, -0.563403308391571, -0.0...
Name: embedding, dtype: object

In [85]:
df['embedding']

0    [-0.25839850306510925, -0.22478339076042175, -...
1    [-0.2057669311761856, -0.6190314888954163, -0....
2    [-0.43562543392181396, -0.5640958547592163, 0....
3    [-0.1016184613108635, -0.563403308391571, -0.0...
Name: embedding, dtype: object

## Save Data with embedding

In [86]:
df.to_csv('DATA/book.csv', index=False)

In [87]:
df = pd.read_csv('DATA/book.csv')
df

,title,document,embedding
0,L'Intelligence artificielle pour les Nuls - Jo...,I\nChapitre 2\nDéfinir le rôle des données\nDA...,"[-0.25839850306510925, -0.22478339076042175, -..."
1,L'Intelligence artificielle pour les Nuls - Jo...,"humains, mais ces chiffres sont déjà remis en ...","[-0.2057669311761856, -0.6190314888954163, -0...."
2,L'Intelligence artificielle pour les Nuls - Jo...,cessé d’être simplement de l’information trait...,"[-0.43562543392181396, -0.5640958547592163, 0...."
3,L'Intelligence artificielle pour les Nuls - Jo...,"logicielles de Google parcourent le Web, saute...","[-0.1016184613108635, -0.563403308391571, -0.0..."


## Similarity

In [88]:
df['embedding'].values

array(['[-0.25839850306510925, -0.22478339076042175, -0.1182456836104393, -0.2774835526943207, -0.26480087637901306, 0.13150541484355927, 0.350213885307312, -0.18685854971408844, -0.10861601680517197, 0.10110661387443542, -0.297537624835968, 0.1369483917951584, 0.08593381941318512, -0.0008559546549804509, -0.35835957527160645, -0.42003166675567627, -0.009533344767987728, 0.1524168998003006, -0.26878291368484497, -0.2179126739501953, 0.2129247635602951, -0.17680135369300842, 0.1384316086769104, 0.3351246416568756, 0.08220383524894714, -0.08939570188522339, -0.4340517222881317, -0.06382901221513748, 0.21157528460025787, -0.10549997538328171, 0.1346093863248825, 0.12916511297225952, 0.221822589635849, -0.4970206022262573, -0.30573877692222595, 0.23057425022125244, 0.07512294501066208, -0.07099546492099762, 0.0105217806994915, 0.04292435571551323, 0.42962729930877686, -0.11645057797431946, 0.01241141464561224, 0.21196229755878448, -0.1554216742515564, 0.2545906901359558, 0.6134476065635681

In [74]:
embeddings = []
for item in df['embedding']:
    embeddings.append(item)
embeddings = np.array(embeddings)
print(embeddings.shape)
embeddings

(4, 1024)


array([[-0.2583985 , -0.22478339, -0.11824568, ..., -0.40508503,
         0.48951012,  0.02105767],
       [-0.20576693, -0.61903149, -0.0362639 , ..., -0.30370915,
         0.33362311,  0.14169537],
       [-0.43562543, -0.56409585,  0.03337699, ..., -0.11691995,
         0.35346362, -0.13668972],
       [-0.10161846, -0.56340331, -0.00586887, ..., -0.13054456,
         0.29764754,  0.09785139]], shape=(4, 1024))

In [76]:
dim = embeddings.shape[1]
assert dim == 1024
print("dim", dim)
index = faiss.IndexFlatIP(dim)
print(type(index))
index.add(embeddings)

dim 1024
<class 'faiss.swigfaiss_avx2.IndexFlatIP'>
